In [1]:
import DBconnect

In [2]:
import csv
import math
from statistics import median

In [12]:
#using Binance
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import pandas as pd
import numpy as np
client = Client('tJaGBlHOgg9cYqQTSxY0R2iudEmZBW6myxGk6SjfBSKRsn0HMFoIseSUmA8hy24I', 'OKtFEor2lJo2uyG7HMqPxUHqCBzzZslGWKW37SAz6ZTpkk319GDpwtpPl2T31iHq', tld='us')


In [4]:
#Getting All symbols from Binance US
#Get 20 symbols per IP to not get blocked
raw_symbols = client.get_all_tickers()
bi_symbols = []

for s in range(len(raw_symbols)):
    bi_symbols.append(raw_symbols[s]['symbol'])


In [5]:
for i in range(3):
    for w in bi_symbols:
        if w.endswith('USD'):
            pass
        else:
            bi_symbols.remove(w)
for i in range(3):
    for w in bi_symbols:
        if w.endswith('BUSD'):
            bi_symbols.remove(w)
        else:
            pass

['BTCUSD', 'ETHUSD', 'XRPUSD', 'BCHUSD', 'LTCUSD', 'USDTUSD', 'ADAUSD', 'BATUSD', 'ETCUSD', 'XLMUSD', 'ZRXUSD', 'LINKUSD', 'DASHUSD', 'ZECUSD', 'ALGOUSD', 'IOTAUSD', 'BUSDUSD', 'WAVESUSD', 'ATOMUSD', 'NEOUSD', 'QTUMUSD', 'NANOUSD', 'ICXUSD', 'ENJUSD', 'ONTUSD', 'ZILUSD', 'VETUSD', 'XTZUSD', 'HBARUSD', 'OMGUSD', 'MATICUSD', 'REPUSD', 'EOSUSD', 'DOGEUSD', 'KNCUSD', 'VTHOUSD', 'USDCUSD', 'COMPUSD', 'MANAUSD', 'HNTUSD', 'MKRUSD', 'DAIUSD', 'ONEUSD', 'BANDUSD', 'STORJUSD', 'UNIUSD', 'SOLUSD', 'EGLDUSD', 'PAXGUSD', 'OXTUSD', 'ZENUSD', 'FILUSD', 'AAVEUSD', 'GRTUSD', 'SUSHIUSD', 'ANKRUSD', 'AMPUSD', 'CRVUSD', 'AXSUSD', 'AVAXUSD', 'CTSIUSD', 'DOTUSD', 'YFIUSD', '1INCHUSD']
64


In [6]:
hltable=pd.DataFrame()

In [20]:
#Get 20 symbols per IP to not get blocked
for bs in range(0,64):
    klines = client.get_historical_klines(bi_symbols[bs], Client.KLINE_INTERVAL_1WEEK, "1 DEC, 2020")
    df = pd.DataFrame(klines,columns = ['OpenTime','Open','High','Low','Close','Volume','CloseTime','QAV','Trades','TBAV','TQAV','ignore'])
    df = df.iloc[::-1]
    if bs == 0:
        hltable['time'] = df['OpenTime']
    hltable[bi_symbols[bs]+"-Close"] = df['Close']
    hltable[bi_symbols[bs]+"-High"] = df['High']
    hltable[bi_symbols[bs]+"-Low"] = df['Low']
    

In [22]:
hltable.dropna(axis='columns', inplace=True)

In [24]:
hlCompareTable=pd.DataFrame()
hlCompareTable2=pd.DataFrame()

In [25]:
#hlCompare Table saves Area, hlcompareTable2 saves High Price 
tlen = hltable.shape[0]
for col in hltable.columns:
    currency = col.split('-')[0]
    if col == 'time':
        continue
    elif currency in hlCompareTable.columns and currency in hlCompareTable2.columns:
        continue
    else:
        valueList = np.zeros(shape=(tlen-1,1))
        secondValueList = np.zeros(shape=(tlen-1,1))
        for i in range(tlen-1):
            try:
                c = float(hltable[currency+'-Close'][tlen-2-i])
            except:
                print(i)
            h = float(hltable[currency+'-High'][tlen-1-i])
            l = float(hltable[currency+'-Low'][tlen-1-i])

            if pd.isna(c) and pd.isna(h) and pd.isna(l):    
                break
            else:
                valueList[i][0] =((h-l)/c*100)
                secondValueList[i][0] =((h-c)/c*100)
        hlCompareTable[currency] = pd.DataFrame(valueList, columns = [currency])[currency]
        hlCompareTable2[currency] = pd.DataFrame(secondValueList, columns = [currency])[currency]
    

In [65]:
# Get table of runners: calculated base price and runner price
# CalcTable saves area difference, CalcTable2 saves high price difference.

hlCalcTable=pd.DataFrame()
hlCalcTable2=pd.DataFrame()

for col in hlCompareTable.columns:
    if col == 'BTCUSD':
        pass
    elif col in hlCalcTable.columns and col in hlCalcTable2.columns:
        pass
    else:
        calcList = np.zeros(shape=(tlen-2,1))
        secondCalcList = np.zeros(shape=(tlen-2,1))
        for i in range(tlen-2):
            baseArea = float(hlCompareTable['BTCUSD'][i])
            runnerArea = float(hlCompareTable[col][i+1])

            baseHigh = float(hlCompareTable2['BTCUSD'][i])
            runnerHigh = float(hlCompareTable2[col][i+1])

            calcList[i][0] = baseArea - runnerArea
            secondCalcList[i][0] = baseHigh - runnerHigh
            
        hlCalcTable[col] = pd.DataFrame(calcList, columns = [col])[col]
        hlCalcTable2[col] = pd.DataFrame(secondCalcList, columns = [col])[col]
    

In [67]:
# Get a table of similarly moved runners

similarTable=pd.DataFrame()
signalCol = []
signalArea = []
signalHigh = []
resultHigh = []
signalIndex = []
for col in hlCalcTable.columns:

    for i in range(1,tlen-2):
        if abs(hlCalcTable[col][i]) < 1:
            if hlCalcTable2[col][i] < 1:
                    signalArea.append(round(hlCompareTable['BTCUSD'][i],2))
                    signalHigh.append(round(hlCompareTable2['BTCUSD'][i],2))
                    resultHigh.append(round(hlCompareTable2[col][i-1],2))
                    signalIndex.append(i)
                    signalCol.append(col)
try:
    similarTable['Signal Col'] = pd.Series(signalCol)
    similarTable['Signal Area'] = pd.Series(signalArea)
    similarTable['Signal High'] = pd.Series(signalHigh)
    similarTable['Result High'] = pd.Series(resultHigh)
    similarTable['Signal Index'] = pd.Series(signalIndex)
except:
        print("An exception occurred")
similarTable = similarTable.loc[similarTable['Result High'] > 1]

In [68]:
#Getting Max signaled currency

pivotTable = similarTable.pivot_table(columns=['Signal Col'], aggfunc='size')
pivotTable.sort_values(ascending=False, inplace =True)
maxSignal = pivotTable.max()

maxSeries = pd.DataFrame(pivotTable.where(pivotTable.eq(maxSignal))).stack()
maxList = []
for i in range(maxSeries.size):
    maxList.append(maxSeries.index[i][0])

In [87]:
# Get most high currency among max signaled currencies
maxUnit = ''
buySignal = False
for m in pivotTable.index:
    if pivotTable[m] < 5:
        break
    indexList = similarTable.where(similarTable['Signal Col'] == m).dropna().index
    signalHighList = []
    signalAreaList = []
    for i in indexList:
        signalHighList.append(similarTable.where(similarTable['Signal Col'] == m).dropna()['Signal High'][i])
        signalAreaList.append(similarTable.where(similarTable['Signal Col'] == m).dropna()['Signal Area'][i])

    signalEvaluateTable = hlCompareTable[['BTCUSD', m]]
    signalEvaluateTable['BTCUSD-High'] = hlCompareTable2['BTCUSD']
    signalEvaluateTable[m+'-High'] = hlCompareTable2[m]

    signalIndexList = []
    for i in range(len(signalHighList)):
        sIndexList = signalEvaluateTable.loc[(signalEvaluateTable['BTCUSD'] > signalAreaList[i]-0.5) & (signalEvaluateTable['BTCUSD'] < signalAreaList[i]+0.5) & (signalEvaluateTable['BTCUSD-High'] < signalHighList[i]+0.5) & (signalEvaluateTable['BTCUSD-High'] < signalHighList[i]+0.5)].index
        for s in sIndexList:
            signalIndexList.append(s)

    #if signalIndexList contains 1 -> buy signal
    # print(set(signalIndexList))
    if signalIndexList.count(1) > 0:
        buySignal = True
        maxUnit = m
        break
    else:
        buySignal = False

<ipython-input-87-46053d45dc45>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signalEvaluateTable['BTCUSD-High'] = hlCompareTable2['BTCUSD']
<ipython-input-87-46053d45dc45>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signalEvaluateTable[m+'-High'] = hlCompareTable2[m]


In [298]:
# Get most high currency among max signaled currencies
maxUnit = ''
maxHigh = 0
for m in maxList:
    avgHigh = similarTable.where(similarTable['Signal Col'] == m).dropna()['Result High'].median()
    if avgHigh > maxHigh:
        maxHigh = avgHigh
        maxUnit = m

In [300]:
indexList = similarTable.where(similarTable['Signal Col'] == maxUnit).dropna().index
signalHighList = []
signalAreaList = []
for i in indexList:
    signalHighList.append(similarTable.where(similarTable['Signal Col'] == maxUnit).dropna()['Signal High'][i])
    signalAreaList.append(similarTable.where(similarTable['Signal Col'] == maxUnit).dropna()['Signal Area'][i])

signalEvaluateTable = hlCompareTable[['BTCUSD', maxUnit]]
signalEvaluateTable[['BTCUSD-High',maxUnit+'-High']] = hlCompareTable2[['BTCUSD', maxUnit]]

signalIndexList = []
for i in range(len(signalHighList)):
    sIndexList = signalEvaluateTable.loc[(signalEvaluateTable['BTCUSD'] > signalAreaList[i]-0.5) & (signalEvaluateTable['BTCUSD'] < signalAreaList[i]+0.5) & (signalEvaluateTable['BTCUSD-High'] < signalHighList[i]+0.5) & (signalEvaluateTable['BTCUSD-High'] < signalHighList[i]+0.5)].index
    for s in sIndexList:
        signalIndexList.append(s)

#if signalIndexList contains 1 -> buy signal
# print(set(signalIndexList))
if signalIndexList.count(1) + signalIndexList.count(0) > 0:
    buySignal = True
else:
    buySignal = False

In [96]:
# Result Median High is expected profit %
sigCount = 0
failcount = 0
sigResultList = []
sigFailList = []
for i in signalIndexList:
    try:
        sigResultList.append(hlCompareTable2[maxUnit][i])
        sigCount+=1
    except:
        pass
    
med = median(sigResultList)

for i in signalIndexList:
    try:
        if hlCompareTable2[maxUnit][i] < round(med,2):
            failcount += 1
            sigFailList.append(float(hltable[maxUnit+'-Close'][tlen-i+1]) - float(hltable[maxUnit+'-Close'][tlen-i]))
    except:
        pass
        
print('Signal Unit: ', maxUnit)        
print('Result median High: ', round(med,2))
print('Fail Median: ', round(median(sigFailList),2))
print('Signal Count : ',len(sigResultList))
print('Fail Count: ', failcount)

Signal Unit:  ALGOUSD
Result median High:  11.65
Fail Median:  0.01
Signal Count :  13
Fail Count:  6


In [4]:
if buySignal:
    db = DBconnect.myDB()
    db.runQuery(maxUnit, round(med,2))
else:
    db = DBconnect.myDB()
    db.cursor.execute('delete from t_signal')